In [27]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime

In [28]:
#신세계푸드 031440 / 신세계건설 034300 / 신세계 I&C 035510
# 광주신세계 037710 / (주식회사)신세계 004170 / (주식회사)이마트 139480 / 현대건설 000720 / CJ제일제당 097950
codes=['031440','034300','035510','037710','004170','139480','000720','097950']

In [29]:
forders = os.listdir('./data/종목데이터/20220707_20230626/')
print(forders[:-8])

['2022_10', '2022_11', '2022_12', '2022_7', '2022_8', '2022_9', '2023_1', '2023_2', '2023_3', '2023_4', '2023_5', '2023_6', '2023_7']


In [30]:
# 데이터 불러오기

# '2022_10', '2022_11', '2022_12', '2022_7', '2022_8', '2022_9', '2023_1', '2023_2', '2023_3', '2023_4', '2023_5', '2023_6'
for forder in forders:

    # '031440','034300','035510','037710','004170','139480','000720','097950'
    for code in codes:

        globals()['df_{}_{}'.format(forder,code)] = pd.DataFrame()

        for file_path in glob.glob('./data/종목데이터/20220707_20230626/{}/{}_*.csv'.format(forder,code)):
           
            df= pd.read_csv(file_path, encoding='utf-8') 
            globals()['df_{}_{}'.format(forder,code)] = pd.concat([globals()['df_{}_{}'.format(forder,code)], df])

        # print('df_{}_{}'.format(forder,code))


In [31]:
# 데이터 종목별로 합치기
# '031440','034300','035510','037710','004170','139480','000720','097950'
for code in codes:

    globals()['df_{}'.format(code)] = pd.DataFrame()
    
    # '2022_10', '2022_11', '2022_12', '2022_7', '2022_8', '2022_9', '2023_1', '2023_2', '2023_3', '2023_4', '2023_5', '2023_6'
    for forder in forders:

        df= globals()['df_{}_{}'.format(forder,code)]
        globals()['df_{}'.format(code)] = pd.concat([globals()['df_{}'.format(code)], df])


In [32]:
for code in codes:

    globals()['df_{}'.format(code)].insert(0,'Datetime', globals()['df_{}'.format(code)]['date'].astype('str') + globals()['df_{}'.format(code)]['time'].astype('str'))

    globals()['df_{}'.format(code)]['Datetime'] = globals()['df_{}'.format(code)]['Datetime'].apply(lambda _ : datetime.strptime(_,'%Y%m%d%H%M%S'))

    globals()['df_{}'.format(code)].sort_values('Datetime',inplace=True,ignore_index=True)

    globals()['df_{}'.format(code)] = globals()['df_{}'.format(code)][['Datetime', 'open', 'high', 'low', 'close', 'qty','value']]

    # 15시20분 이후 데이터 자르기
    i=0
    for time_ in globals()['df_{}'.format(code)]['Datetime'].apply(lambda _ : _.time()):
        if time_ > datetime.strptime('1520','%H%M').time():
            globals()['df_{}'.format(code)].drop(index=[i],axis=1,inplace=True)
            i+=1
        else:
            i+=1


In [33]:
for code in codes:

    dates = globals()['df_{}'.format(code)]['Datetime'].dt.date.unique()

    globals()['df2_{}'.format(code)]= pd.DataFrame()

    for date in dates:
        daily_data = globals()['df_{}'.format(code)][globals()['df_{}'.format(code)]['Datetime'].dt.date == date].copy()

        # 9시~3시20분 1분 타임라인 리스트
        timelist = pd.date_range('09:01','15:20',freq='1min').time.astype('str')

        # 빠진 시간
        abtimes =  list(set(timelist)-set(daily_data['Datetime'].apply(lambda _ : _.time().strftime('%H:%M:%S'))))

        datetimes = []
        for ab in abtimes:
            datetimes.append(datetime.strptime(date.strftime('%Y%m%d')+ ab,'%Y%m%d%H:%M:%S'))
            
        testdf = pd.DataFrame(columns=daily_data.columns)
        testdf['Datetime'] = datetimes


        daily_data = pd.concat([daily_data,testdf])

        daily_data.sort_values(by='Datetime', ascending=True, inplace=True)

        globals()['df2_{}'.format(code)] = pd.concat([globals()['df2_{}'.format(code)],daily_data])

In [34]:
df2_031440

,Datetime,open,high,low,close,qty,value
0,2022-07-07 09:01:00,57300,57900,57300,57900,3,0
1,2022-07-07 09:02:00,57900,57900,57900,57900,1,0
2,2022-07-07 09:03:00,57800,57900,57800,57900,3,0
3,2022-07-07 09:04:00,57300,57900,57300,57900,2,0
4,2022-07-07 09:05:00,57800,57800,57800,57800,1,0
...,...,...,...,...,...,...,...
98232,2023-07-20 15:16:00,41100,41100,41100,41100,0,0
98233,2023-07-20 15:17:00,41100,41100,41100,41100,0,0
98234,2023-07-20 15:18:00,41100,41200,41100,41200,32,1
98235,2023-07-20 15:19:00,41200,41200,41200,41200,0,0


In [35]:
for code in codes:
    globals()['df2_{}'.format(code)]['value'] = globals()['df2_{}'.format(code)]['open'] * globals()['df2_{}'.format(code)]['qty']
    globals()['df2_{}'.format(code)].reset_index(inplace=True,drop=True)

    globals()['df2_{}'.format(code)]['open'].ffill(inplace=True)
    globals()['df2_{}'.format(code)]['high'].ffill(inplace=True)
    globals()['df2_{}'.format(code)]['low'].ffill(inplace=True)
    globals()['df2_{}'.format(code)]['close'].ffill(inplace=True)
    globals()['df2_{}'.format(code)]['qty'].fillna(value=0, inplace=True)
    globals()['df2_{}'.format(code)]['value'].fillna(value=0, inplace=True)

    globals()['df2_{}'.format(code)].drop_duplicates(['Datetime'], keep='first', inplace=True, ignore_index=True)
    
    globals()['df2_{}'.format(code)].to_csv('./data/종목데이터/20220707_20230626/df_{}.csv'.format(code),index=False)

In [36]:
df2_000720

,Datetime,open,high,low,close,qty,value
0,2022-07-07 09:01:00,38700,38900,38700,38850,11811,457085700
1,2022-07-07 09:02:00,38850,38900,38650,38750,7519,292113150
2,2022-07-07 09:03:00,38750,38800,38650,38800,8415,326081250
3,2022-07-07 09:04:00,38800,38950,38800,38900,5021,194814800
4,2022-07-07 09:05:00,38900,38900,38700,38750,2731,106235900
...,...,...,...,...,...,...,...
98035,2023-07-20 15:16:00,37450,37450,37400,37400,2456,91977200
98036,2023-07-20 15:17:00,37450,37450,37250,37350,13825,517746250
98037,2023-07-20 15:18:00,37350,37400,37300,37400,1349,50385150
98038,2023-07-20 15:19:00,37350,37400,37300,37400,1007,37611450
